# Step by step segmentation example
This example is meant to show how to main function works and how the intermediate results relate to each other. This is not supposed to be an example of how the package is actually used in practice. Here, intermediate results of the segmentation process are shown, whereas in practice (using the `pipeline` function) nothing similar is shown (though some intermediate results are saved and can be reviewed).

To actually run this example, one .nd2 image sequence file is needed. This file is not provided with the package as these are usually very large.

First, we will import the `bcells` package and also some other packages that we will need later. We will also set some options to be able to view plots.

Cells containing code will run if the desired cell is marked and then `Shift+Enter` is pressed (if the keyboard shortcuts have not been changed). Alternatively, there should also be a run button which can be clicked so that the marked cell is run.

In [1]:
import bcells
import os
import nd2
import numpy as np
from skimage.color import label2rgb
import matplotlib.pyplot as plt
%matplotlib inline

Now, we will need to load the actual data with the .nd2 file. We will need the correct path to one such file. You will need to adjust the path to the file to match your system and setup. 

One easy way to do this would be to copy this jupyter notebook file we are currently running to the same folder as the .nd2 file. In that case, only the file name would be needed.

You will need to uncomment some lines below and use the appropriate path to the file and the right filename. Uncommenting can be done by removing the `#` character from the beginning of the line. For the file name, the third line in the following cell should be uncommented and the correct file name should be entered between the quotation marks instead of `example_file.nd2`.

In [2]:
# path_to_folder = '/data/' # e.g. when the .nd2 file is in the subfolder 'data' relative to this jupyter notebook
path_to_folder = '.' # ADJUST # '.' is correct if .nd2 file is in the same folder as this jupyter notebook
filename = 'example_10muM_000.nd2' # ADJUST
path_to_nd2_file = os.path.join(path_to_folder, filename)

print(path_to_nd2_file) # print the path

../../data/20220218/NIS Acquisition Data/20220218_ADHEK293-OptoRyR2_Ex2_Tetracaine_0 µM_046.nd2


We can check if we are in the right folder by listing all files in that folder. Furthermore, we can check if the file we are looking for is in that list.

In [3]:
# list files in folder
print(os.listdir(path_to_folder))
print('The file {} is contained in the folder: {}'.format(filename, filename in os.listdir(path_to_folder)))

['20220218_ADHEK293-OptoRyR2_Ex2_Tetracaine_5 µM_025.nd2', '20220218_ADHEK293-OptoRyR2_Ex2_Tetracaine_500 µM_040.nd2', '20220218_ADHEK293-OptoRyR2_Ex2_Tetracaine_1 µM_023.nd2', '20220218_ADHEK293-OptoRyR2_Ex2_Tetracaine_1 mM_042.nd2', '20220218_ADHEK293-OptoRyR2_Ex2_Tetracaine_50 µM_036.nd2', 'Thumbs.db', '20220218_ADHEK293-OptoRyR2_Ex2_Tetracaine_10 µM_027.nd2', '20220218_ADHEK293-OptoRyR2_Ex2_Tetracaine_100 µM_038.nd2', '20220218_ADHEK293-OptoRyR2_Ex2_Tetracaine_0 µM_046.nd2', '20220218_ADHEK293-OptoRyR2_Ex2_Tetracaine_0.1 µM_031.nd2', '20220218_ADHEK293-OptoRyR2_Ex2_Tetracaine_0.5 µM_029.nd2']
The file 20220218_ADHEK293-OptoRyR2_Ex2_Tetracaine_0 µM_046.nd2 is contained in the folder: True


Equipped with the path, we can actually load the file. This might take roughly 30 seconds.

In [4]:
# load the nd2 file
imgs = nd2.imread(path_to_nd2_file)

If there is an error with the import, the most likely reason is that something is wrong with the path to the file.

Now, show the first frame of the image sequence.

In [ ]:
fig, ax = plt.subplots()
ax.imshow(imgs[0, :, :], cmap='gray')
ax.set_title("First frame")
ax.set_axis_off()
plt.tight_layout()
plt.show()

Calculate a denoised version of an averaged version of some frames and show the results afterwards. This might take up to 2 minutes.

In [8]:
denoised, _ = bcells.processing.process.average_and_denoise(imgs, frames=slice(120, 280))

In [ ]:
fig, ax = plt.subplots()
ax.imshow(denoised, cmap='gray')
ax.set_title("Denoised")
ax.set_axis_off()
plt.tight_layout()
plt.show()

Now, we clip the brightest pixel to prevent one or a few cells being much brighter than most other ones having a big effect. Then, we estimate the background intensity. We plot the estimated background intensity and the current image in this data pipeline where we substracted the estimated background. This might take up to 15 seconds.

In [10]:
clipped = bcells.processing.process.clip_and_rescale(denoised)
c_minus_bg, est_bg = bcells.processing.process.subtract_local_bg(clipped, offset = -0.01)

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
axs[0].imshow(est_bg, cmap='gray')
axs[0].set_title("Estimated background")
axs[0].set_axis_off()
axs[1].imshow(c_minus_bg, cmap='gray')
axs[1].set_title("Background substracted")
axs[1].set_axis_off()
plt.tight_layout()
plt.show()

Get local thresholds of the processed image and show the not so strict and strict thresholds. This might take up to 20 seconds.

In [12]:
local_thresh_img, local_thresh_markers, _, _, _ = bcells.processing.process.local_threshold(c_minus_bg)

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2)
axs[0].imshow(local_thresh_img, cmap='gray')
axs[0].set_title("Local threshold")
axs[0].set_axis_off()
axs[1].imshow(local_thresh_markers, cmap='gray')
axs[1].set_title("Strict local threshold")
axs[1].set_axis_off()
plt.tight_layout()
plt.show()

Finally, we compute the segmentation using the thresholded images. Then, we do some slight processing of the output and show those.

In [14]:
seg_out = bcells.segmentation.segment.segment(img=local_thresh_img, markers=local_thresh_markers)
seg = bcells.segmentation.segment.process_segmentation(seg_out)

In [ ]:
seg_out_color = label2rgb(seg_out, bg_label=0)
seg_color = label2rgb(seg, bg_label=0)
fig, axs = plt.subplots(nrows=1, ncols=2)
axs[0].imshow(seg_out_color)
axs[0].set_title("Segmentation")
axs[0].set_axis_off()
axs[1].imshow(seg_color)
axs[1].set_title("Segmentation (processed)")
axs[1].set_axis_off()
plt.tight_layout()
plt.show()

We compute the brightness of the detected cells over the image sequence. Additionally, we calculate local background intensity traces of each cell and finally the intensity traces of the cells where we substracted the calculated local background intensity. This might take up to a minute.

In [16]:
imgs = np.transpose(imgs, (1, 2, 0)) # regionprops_table expects last dimension to be time dimension
seg_props = bcells.segmentation.extract.seg_properties(seg_lbl=seg, imgs=imgs)

In [17]:
seg_local_bg_cells = bcells.segmentation.extract.local_bg_intensity(imgs=imgs, seg_bg=(seg == 0).astype(np.int8), cells_slice=seg_props['slice'])
del imgs

In [18]:
traces_bg = bcells.segmentation.extract.traces_minus_bg(seg_props, seg_local_bg_cells)

Plot resulting traces.

In [ ]:
fig, ax = plt.subplots()
ax.plot(traces_bg.T)
ax.set_title("Traces")
plt.show()